In [24]:
import os
import itertools

import pandas as pd
import xarray as xr

import climatools.lblnew.pipeline as pipeline
import climatools.lblnew.setup_bestfit as setup_bestfit
import climatools.lblnew.bestfit_params as bestfit

import importlib
importlib.reload(setup_bestfit)

<module 'climatools.lblnew.setup_bestfit' from '/chia_cluster/home/jackyu/climatools/climatools/lblnew/setup_bestfit.py'>

In [25]:
def define_param():
    '''
    Define a particular lblnew best-fit run.
    '''
    molecule = 'h2o'
    band = '1'
    dv = .005
    nv = 200
    commitnumber = '9d81085'
    
    atmpro = 'mls'
    tsfc = 294
    
    param = bestfit.kdist_params(molecule='h2o', band='1')
    param['molecule'] = molecule
    param['band'] = band
    param['dv'] = dv
    param['nv'] = nv
    param['commitnumber'] = commitnumber
    param['atmpro'] = atmpro
    param['tsfc'] = tsfc
    return param



def fpath(param):
    path = os.path.join(pipeline.get_dir_case(param, 
                                               setup=setup_bestfit),
                         'dgdgs.dat')
    return path



def load_dgdgs(path):
    df = pd.read_csv(path, sep=r'\s+')

    df = df.set_index('g')    
    return df



def into_chunks(l, chunksize):
    return itertools.zip_longest(*(chunksize * [iter(l)]))



def vector_to_F77list(array, num_values_per_line=4):
    '''
    '''
    strfmt = '{:15.6e}'
    
    chunks = into_chunks(array, num_values_per_line)
    
    chunks = list(chunks)
    
    lines = []
    for chunk in chunks[:-1]:
        vs = [strfmt.format(v) for v in chunk if v != None]
        line = ','.join(vs)
        line = line + ','
        lines.append(line)
        
    vs = [strfmt.format(v) for v in chunks[-1] if v != None]
    line = ','.join(vs)
    lines.append(line)
    
    return lines



def dgdgs_to_F77(dgdgs):
    lines = vector_to_F77list(dgdgs, num_values_per_line=3)
    
    rlines = []
    rlines.append(5 * ' ' + '&/' + lines[0])
    for line in lines[1:-1]:
        rlines.append(5 * ' ' + '&' + line)
    rlines.append(5 * ' ' + '&' + lines[-1] + '/')
    
    fortran = '\n'.join(rlines)
    return fortran


def script():
    dgdgs = load_dgdgs(fpath(define_param()))
    return dgdgs_to_F77(dgdgs['dgs'])

In [26]:

print(script())


     &/   2.470455e-04,   5.019937e-04,   1.207687e-03,
     &   2.843600e-03,   9.518264e-03,   2.234796e-02,
     &   7.766095e-02,   2.223594e-01,   3.632112e-01,
     &   3.001019e-01/
